# Benchmark Brax Rendering Example using JaxRenderer backend

Self-link: https://colab.research.google.com/drive/1xhkYNz5WjvUCjQWpp72CLf9SIy3i5PnN

Benchmarking using A100: https://colab.research.google.com/drive/1A7PzhG3vn6oNzrWTxE5E3dmu8xQTcNnH

Benchmarking using A100 on smaller canvas: https://colab.research.google.com/drive/1Mr2pRetdobZdgdtNT_DJuS4YZq3-C4oY

Generate data: https://colab.research.google.com/drive/1c_83TLtb_pOt4OSlWFQgIKzA9DxdBDTp

<details>
  <summary>Other related profiling/benchmarking Colabs</summary>

Profile with pre-generated data: https://colab.research.google.com/drive/12yNBVOdwUqUOBRgmQHF0gl8eMzzIi_BH

Previous profile version (profile branch head): https://colab.research.google.com/drive/1V7gdTY6ZYz7YhJI_LCWLZM035zSDTYoQ

Previous prfiling version: https://colab.research.google.com/drive/1BJQG38IHPaUuMKMnNeq0hjrLtqwx4OfU

All-inlines + minibatch loops: https://colab.research.google.com/drive/1NiCTOCxfU0Mvr818Zqt4y63-S20Gn8yb

All-inlines: https://colab.research.google.com/drive/1Wrt3a0yoVPhYUJYCZltFIYoELWD-ciE0

Baseline link: https://colab.research.google.com/drive/17NSjyJL_Ov9D32Mnrs7miZmjiPNf9U6l

</details>

The sample output is using T4 backend, with standard RAM.

In [ ]:
#@title Reinstall jaxrenderer from tag `v0.3.0`; Install pytinyrenderer
!pip uninstall jaxrenderer -y -qqq
!pip install git+https://github.com/JoeyTeng/jaxrenderer.git@v0.3.0 -qqq

!pip install pytinyrenderer --upgrade -qqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00


In [ ]:
#@title ## GPU Spec
!nvidia-smi

Mon Jun 12 12:20:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title ## Imports
#@markdown **⚠️ PLEASE NOTE:**

#@markdown This colab runs best using a GPU runtime.  From the Colab menu, choose Runtime > Change Runtime Type, then select **'GPU'** in the dropdown.

from functools import partial
from typing import Optional
import os
import pickle

import jax
from jax import numpy as jp
import numpy as onp
import pandas as pd
from scipy.spatial.transform import Rotation as R

import pytinyrenderer

from renderer import CameraParameters as Camera
from renderer import LightParameters as Light
from renderer import ModelObject as Instance
from renderer import ShadowParameters as Shadow
from renderer import Renderer, transpose_for_display

if 'COLAB_TPU_ADDR' in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

In [ ]:
#@title ## Load data; Constants
!rm inputs-30.zip inputs.pickle
!wget https://github.com/JoeyTeng/jaxrenderer/raw/92904b74f4d760cd66b8940ed41c2d854cfbebe9/test_resources/pre-gen-brax/inputs-30.zip
!unzip inputs-30.zip

_batched_instances, _batched_camera, _batched_target = pickle.load(open("inputs.pickle", "rb"))

canvas_width: int = 960 #@param {type:"integer"}
canvas_height: int = 540 #@param {type:"integer"}
number_of_frames_to_render: int = 2 #@param {type:"slider", min: 1, max: 30, step: 1}

batched_instances = jax.tree_map(lambda a: jp.asarray(a[:number_of_frames_to_render]).block_until_ready(), _batched_instances)
batched_camera = jax.tree_map(lambda a: jp.asarray(a[:number_of_frames_to_render]).block_until_ready(), _batched_camera)
batched_target = jax.tree_map(lambda a: jp.asarray(a[:number_of_frames_to_render]).block_until_ready(), _batched_target)

rm: cannot remove 'inputs-30.zip': No such file or directory
rm: cannot remove 'inputs.pickle': No such file or directory
--2023-06-12 12:21:02--  https://github.com/JoeyTeng/jaxrenderer/raw/92904b74f4d760cd66b8940ed41c2d854cfbebe9/test_resources/pre-gen-brax/inputs-30.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JoeyTeng/jaxrenderer/92904b74f4d760cd66b8940ed41c2d854cfbebe9/test_resources/pre-gen-brax/inputs-30.zip [following]
--2023-06-12 12:21:02--  https://raw.githubusercontent.com/JoeyTeng/jaxrenderer/92904b74f4d760cd66b8940ed41c2d854cfbebe9/test_resources/pre-gen-brax/inputs-30.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP req

In [ ]:
#@title ## Utility Function
import gc
import itertools
import time
from typing import Any, Callable


def benchmark(
    f: Callable[[], Any],
    repeat: int = 2,
    number: int = 1,
    sort: bool = True,
) -> list[int]:
  """Execute given pure function `f` `number` times and take sum, repeat for
    `repeat` times.

  In total, f is executed repeat * number times.

  This function is a very similar re-implementation of `timeit.repeat`.

  Parameters:
    - f: Pure function to be tested, taking no parameter and returns nothing.
      Return value is ignored. This function must be safe to reentrancy.
    - repeat: int, default 2.
    - number: int, default 1.
    - sort: bool, default True. Whether to sort result timings (in ascending
      order)

  Return:
    a list of total execution times, in nanoseconds. The size of the results
    equals to `repeat`.
  """
  result: list[int] = []

  for _ in range(repeat):
    it = itertools.repeat(None, number)
    gcold = gc.isenabled()
    gc.disable()

    try:
      _t_before: int = time.perf_counter_ns()
      for _ in it:
        f()
      _t_after: int = time.perf_counter_ns()
    finally:
      if gcold:
        gc.enable()

    result.append(_t_after - _t_before)

  if sort:
    result.sort()

  return result

## Baseline - Pytinyrenderer (CPU only)

In [ ]:
#@title Prepare data
def prepare_data(
  number_of_frames_to_render: int,
  batched_instances,
  batched_camera,
  batched_target,
):
  _py_scenes = [pytinyrenderer.TinySceneRenderer() for _ in range(number_of_frames_to_render)]
  _py_instances = []
  _py_cameras = []
  _py_targets = onp.asarray(batched_target).tolist()

  # for each batch
  for i in range(number_of_frames_to_render):
    # process camera
    _camera = jax.tree_map(lambda field: field[i], batched_camera)
    camera = pytinyrenderer.TinyRenderCamera(
        viewWidth=int(_camera.viewWidth),
        viewHeight=int(_camera.viewHeight),
        near=float(_camera.near),
        far=float(_camera.far),
        hfov=float(_camera.hfov),
        vfov=float(_camera.vfov),
        position=onp.asarray(_camera.position).tolist(),
        target=onp.asarray(_camera.target).tolist(),
        up=onp.asarray(_camera.up).tolist(),
    )
    _py_cameras.append(camera)
    # process instances
    instances = jax.tree_map(lambda field: field[i], batched_instances)
    instances_ids = []
    for instance in instances:
      _texture = onp.asarray(instance.model.diffuse_map)[:, ::-1, :].swapaxes(0, 1)

      _model_id = _py_scenes[i].create_mesh(
          onp.asarray(instance.model.verts).flatten().tolist(),  # vertices
          onp.asarray(instance.model.norms).flatten().tolist(),  # normals
          onp.asarray(instance.model.uvs, dtype=float).flatten().tolist(),  # uvs
          onp.asarray(instance.model.faces, dtype=int).flatten().tolist(),  # indices
          (onp.asarray(_texture) * 255).astype(int).flatten().tolist(),  # texture
          _texture.shape[0],  # texture_width
          _texture.shape[1],  # texture_height
          1.,  # texture_scaling
      )
      _instance_id = _py_scenes[i].create_object_instance(_model_id)
      _py_scenes[i].set_object_position(_instance_id, onp.asarray(instance.transform[:3, 3]).tolist())
      _py_scenes[i].set_object_orientation(_instance_id, onp.asarray(R.from_matrix(instance.transform[:3, :3]).as_quat()).tolist())
      _py_scenes[i].set_object_local_scaling(_instance_id, onp.asarray(instance.local_scaling).tolist())

      instances_ids.append(_instance_id)

    _py_instances.append(instances_ids)

  return _py_scenes, _py_instances, _py_cameras, _py_targets


prepare = partial(
    prepare_data,
    batched_instances=_batched_instances,
    batched_camera=_batched_camera,
    batched_target=_batched_target,
)

In [ ]:
#@title Benchmarking
num_frames = list(range(1, 31))

def py_render(scenes, instances, cameras, targets):
  for scene, instances, camera, target in zip(scenes, instances, cameras, targets):
    light = pytinyrenderer.TinyRenderLight(shadowmap_center=target)
    _ = scene.get_camera_image(instances, light, camera)

timings = onp.zeros((len(num_frames), 2), dtype=int)

for i, num_frame in enumerate(num_frames):
  scenes, instances, cameras, targets = prepare(num_frame)
  ts = benchmark(lambda: py_render(scenes, instances, cameras, targets))
  timings[i][0] = ts[0]
  timings[i][1] = ts[-1]


pd.DataFrame(timings, index=num_frames, columns=("min", "max"))

,min,max
1,504060638,899243809
2,952465938,976070518
3,950066929,954473320
4,1278338216,1281635547
5,1597911073,1846474602
6,1923835371,1979703459
7,2263217720,2318032512
8,2560599112,2582028790
9,2945657038,3213274670
10,3240611104,3508582042


import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_2457055225387017910, *['index'], **{})
chart

import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_2457055225387017910, *['min'], **{})
chart

import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_2457055225387017910, *['max'], **{})
chart

import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_2457055225387017910, *['index'], **{})
chart

import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_2457055225387017910, *['min'], **{})
chart

import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_2457055225387017910, *['max'], **{})
chart

import altair as alt
from google.colab import autoviz
df_2457055225387017910 = autoviz.get_registered_df('df_2457055225387017910')

def linked_scatter_plots(
    df, colname_pairs, color='steelblue', deselected_color='lightgray',
    opacity=0.8, deselected_opacity=0.4, width=150, height=150):
  interval = alt.selection_interval()
  scatter = (alt.Chart(df).mark_circle()
             .encode(
                 color=alt.condition(
                     interval, alt.value(color), alt.value(deselected_color)),
                 opacity=alt.condition(
                     interval, alt.value(opacity),
                     alt.value(deselected_opacity)))
             .properties(
                 selection=interval, width=width, height=height))
  return alt.hconcat(*[scatter.encode(x=x, y=y) for x, y in colname_pairs])

chart = linked_scatter_plots(df_2457055225387017910, *[[['index', 'min'], ['min', 'max']]], **{})
chart

## Experiment - JaxRenderer

In [ ]:
#@title ### Render function
def profile_compile(batched_instances, batched_camera, batched_target, loop_unroll: int = 1):
  """Return cost analysis. states is batched."""
  @jax.default_matmul_precision("float32")
  def render_instances(
    instances: list[Instance],
    width: int,
    height: int,
    camera: Camera,
    light: Optional[Light] = None,
    shadow: Optional[Shadow] = None,
    camera_target: Optional[jp.ndarray] = None,
    enable_shadow: bool = True,
  ) -> jp.ndarray:
    """Renders an RGB array of sequence of instances.

    Rendered result is not transposed with `transpose_for_display`; it is in
    floating numbers in [0, 1], not `uint8` in [0, 255].
    """
    if light is None:
      direction = jp.array([0.57735, -0.57735, 0.57735])
      light = Light(
          direction=direction,
          ambient=0.8,
          diffuse=0.8,
          specular=0.6,
      )
    if shadow is None and enable_shadow:
      assert camera_target is not None, 'camera_target is None'
      shadow = Shadow(centre=camera_target)
    elif not enable_shadow:
      shadow = None

    img = Renderer.get_camera_image(
      objects=instances,
      light=light,
      camera=camera,
      width=width,
      height=height,
      shadow_param=shadow,
      loop_unroll=loop_unroll,
    )
    arr = jax.lax.clamp(0., img, 1.)

    return arr

  def _render(instances, camera, target) -> jp.ndarray:
    _render = jax.jit(
      render_instances,
      static_argnames=("width", "height", "enable_shadow"),
      inline=True,
    )
    img = _render(instances=instances, width=canvas_width, height=canvas_height, camera=camera, camera_target=target)
    arr = transpose_for_display((img * 255).astype(jp.uint8))

    return arr

  _render_batch = jax.jit(jax.vmap(jax.jit(_render, inline=True)))
  _render_batch_lowered = _render_batch.lower(batched_instances, batched_camera, batched_target)
  _render_batch_compiled = _render_batch_lowered.compile()

  return _render_batch_compiled

### Branchless + loop unroll

In [ ]:
#@title loop unroll options
loop_unroll_cases: list[int] = [1, 2, 3, 4, 16, 64, 192]  # 192 = 960 / 5

In [ ]:
#@title benchmarking
timings = onp.zeros((len(loop_unroll_cases), 4), dtype=int)
column_indices = ["compilation (min)", "compilation (max)", "execution (min)", "execution (max)"]
row_indices = loop_unroll_cases.copy()

for i, loop_unroll in enumerate(loop_unroll_cases):
  ts = benchmark(lambda: profile_compile(batched_instances, batched_camera, batched_target, loop_unroll=loop_unroll))
  # only take the minimum/maximum time
  timings[i, 0] = ts[0]
  timings[i, 1] = ts[-1]

  _render_batch_compiled = profile_compile(batched_instances, batched_camera, batched_target, loop_unroll=loop_unroll)

  ts = benchmark(lambda: _render_batch_compiled(batched_instances, batched_camera, batched_target).block_until_ready())
  # only take the minimum/maximum time
  timings[i, 2] = ts[0]
  timings[i, 3] = ts[-1]

In [ ]:
pd.DataFrame(timings, index=row_indices, columns=column_indices)

,compilation (min),compilation (max),execution (min),execution (max)
1,5467373775,8723981203,1010637901,1087233714
2,5696650694,9056890380,1164108205,1186810148
3,6634441999,9262590665,1212112185,1249134404
4,7883848182,9803042454,1250310109,1282990038
16,17044530316,22141921383,1341551028,1365266285
64,50732766653,55819072241,1103941233,1133499593
192,171307206135,172400763773,1107764211,1158804208


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


### Unroll = 1, number of frames

In [ ]:
#@title Benchmarking
loop_unroll: int = 1
num_frames = list(range(1, 31))

timings = onp.zeros((len(num_frames), 4), dtype=int)
column_indices = ["compilation (min)", "compilation (max)", "execution (min)", "execution (max)"]
row_indices = num_frames.copy()

for i, num_frame in enumerate(num_frames):
  # prepare data
  batched_instances = jax.tree_map(lambda field: field[:num_frame].block_until_ready(), _batched_instances)
  batched_camera = jax.tree_map(lambda field: field[:num_frame].block_until_ready(), _batched_camera)
  batched_target = jax.tree_map(lambda field: field[:num_frame].block_until_ready(), _batched_target)

  # benchmark
  ts = benchmark(lambda: profile_compile(batched_instances, batched_camera, batched_target, loop_unroll=loop_unroll))
  # only take the minimum/maximum time
  timings[i, 0] = ts[0]
  timings[i, 1] = ts[-1]

  _render_batch_compiled = profile_compile(batched_instances, batched_camera, batched_target, loop_unroll=loop_unroll)

  ts = benchmark(lambda: _render_batch_compiled(batched_instances, batched_camera, batched_target).block_until_ready())
  # only take the minimum/maximum time
  timings[i, 2] = ts[0]
  timings[i, 3] = ts[-1]


pd.DataFrame(timings, index=row_indices, columns=column_indices)

,compilation (min),compilation (max),execution (min),execution (max)
1,6860351172,8554546460,619630879,1685503963
2,6436860839,6610481931,1026871858,1058723464
3,6830068403,7610962800,1512081420,1555225907
4,7061889242,7589758905,1987420709,2016301453
5,6437426957,6758433687,2451103774,2497380112
6,6111870354,6775376987,2918876856,2950610473
7,5582391475,7064312607,3393478118,3444540984
8,5429528068,7663224917,3855982847,3879172693
9,6862199025,8131761888,4309888374,4332980811
10,5339570575,7556826644,4797336992,4830205369


import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_6528825560875141711, *['index'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_6528825560875141711, *['compilation (min)'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_6528825560875141711, *['compilation (max)'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def histogram(df, colname, maxbins=20, width=100, height=50):
  return (alt.Chart(df).mark_bar()
          .encode(
              x=alt.X(colname, bin=alt.Bin(maxbins=maxbins)),
              y=alt.Y('count()', axis=alt.Axis(title='count')),
          )
          .properties(width=width, height=height))  #  Pixels.

chart = histogram(df_6528825560875141711, *['execution (min)'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_6528825560875141711, *['index'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_6528825560875141711, *['compilation (min)'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_6528825560875141711, *['compilation (max)'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def value_plot(df, y, sort_ascending=False, width=100, height=50):
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  return (alt.Chart(df.reset_index()).mark_line()
          .encode(x=alt.X('index', title=''), y=alt.X(y, title='value'))
          .properties(width=width, height=height, title=y))

chart = value_plot(df_6528825560875141711, *['execution (min)'], **{})
chart

import altair as alt
from google.colab import autoviz
df_6528825560875141711 = autoviz.get_registered_df('df_6528825560875141711')

def linked_scatter_plots(
    df, colname_pairs, color='steelblue', deselected_color='lightgray',
    opacity=0.8, deselected_opacity=0.4, width=150, height=150):
  interval = alt.selection_interval()
  scatter = (alt.Chart(df).mark_circle()
             .encode(
                 color=alt.condition(
                     interval, alt.value(color), alt.value(deselected_color)),
                 opacity=alt.condition(
                     interval, alt.value(opacity),
                     alt.value(deselected_opacity)))
             .properties(
                 selection=interval, width=width, height=height))
  return alt.hconcat(*[scatter.encode(x=x, y=y) for x, y in colname_pairs])

chart = linked_scatter_plots(df_6528825560875141711, *[[['index', 'compilation (min)'], ['compilation (min)', 'compilation (max)'], ['compilation (max)', 'execution (min)'], ['execution (min)', 'execution (max)']]], **{})
chart

In [ ]:
#@title # Terminating Colab Automatically
try:
  from google.colab import runtime
  print("Terminating Colab")
  runtime.unassign()
except ModuleNotFoundError:
  print("Not in Colab, skip termination")